In [13]:
import pandas as pd, geopandas as gpd, json, numpy as np
import networkx as nx, matplotlib.pyplot as plt, matplotlib as mpl, contextily as ctx
from shapely import wkt
from matplotlib.colors import LinearSegmentedColormap
from pathlib import Path

In [2]:
import gurobipy as gp, pandas as pd
from collections import defaultdict

# ───────────────────────── 0. INPUT
T  = 10
failed_edges = {"e3","e12"}
max_wait = 2
w1, w2, w3 = 1000, 50, 10
CAPACITY = 3
BIG_M = 10**6
SINK = "SINK"                           

edges = {
    "e1": ("n1","n2",1), "e2": ("n2","n3",1),
    "e3": ("n1","n4",2), "e4": ("n4","n3",2),
    "e5": ("n1","n5",2), "e6": ("n5","n6",1),
    "e7": ("n6","n7",1), "e8": ("n7","n3",1),
    "e9": ("n4","n2",1), "e10": ("n2","n1",1), 
    "e11": ("n3","n2",1), "e12": ("n4","n1",2), 
    "e13": ("n3","n4",2), "e14": ("n5","n1",2), 
    "e15": ("n6","n5",1), "e16": ("n7","n6",1), 
    "e17": ("n3","n7",1), "e18": ("n2","n4",1),
}

routes_nodes = {
    "T1":["n1","n2","n3"],
    "T2":["n1","n4","n3"],
    "T3":["n1","n5","n6","n7","n3"],
}
demand = {
    "T1":[("n1","n2",30),("n1","n3",50),("n2","n3",80)],
    "T2":[("n1","n4",50),("n1","n3",60),("n4","n3",100)],
    "T3":[("n1","n5",800),("n1","n6",60),("n1","n7",75),
          ("n1","n3",100),("n5","n6",25),("n5","n7",35),
          ("n5","n3",70),("n6","n7",15),("n6","n3",25),
          ("n7","n3",35)],
}
trains = list(routes_nodes)

nodes = set(n for _, (s,d,_) in edges.items() for n in (s,d))
nodes.add(SINK)                          # ★ SINK 노드 포함

# ───────────────────────── 0-a. 시간확장 아크
arc_list = []                            # (from, to, eid, τ)

# 실제 이동 arc
for eid,(src,dst,tau) in edges.items():
    if eid in failed_edges: continue
    for t in range(T + 1 - tau):
        arc_list.append((f"{src}^{t}", f"{dst}^{t+tau}", eid, tau))

# wait-arc
for n in nodes - {SINK}:
    for w in range(1, max_wait+1):
        for t in range(T + 1 - w):
            arc_list.append((f"{n}^{t}", f"{n}^{t+w}", f"w_{n}_{w}", w))

# ★ dummy-arc : 후보 종착지 n^t → SINK^t  (τ=0)
for n in nodes - {SINK}:
    for t in range(T+1):
        arc_list.append((f"{n}^{t}", f"{SINK}^{t}", f"dummy_{n}", 0))

df_arc = pd.DataFrame(arc_list, columns=["from","to","eid","tau"])

# ───────────────────────── 0-b. 예정 도착시각
sched = {}
for tr, path in routes_nodes.items():
    t = 0; arr = {path[0]: 0}
    for i in range(len(path)-1):
        u,v = path[i], path[i+1]
        eid = next(e for e,(s,d,_) in edges.items() if s==u and d==v)
        t += edges[eid][2]
        arr[v] = t
    sched[tr] = arr

q_r = {tr: sum(q for *_,q in demand[tr]) for tr in trains}

# ───────────────────────── 1. 모델
m = gp.Model(); m.Params.OutputFlag = 0

# 1-a. 변수
x = {(tr, fr, to, eid): m.addVar(vtype=gp.GRB.BINARY)
     for (fr,to,eid,_) in arc_list for tr in trains}

h = {tr: m.addVar(vtype=gp.GRB.BINARY) for tr in trains}

r_T = {tr: set(routes_nodes[tr][1:]) for tr in trains}
y = {(tr,n): m.addVar(vtype=gp.GRB.BINARY)
     for tr in trains for n in r_T[tr]}

s = {(tr,n): m.addVar(vtype=gp.GRB.BINARY)
     for tr in trains for n in nodes - {SINK}}

z = {(tr,o,d): m.addVar(vtype=gp.GRB.BINARY)
     for tr in trains for (o,d,_) in demand[tr]}

delta = {(tr,o,d): m.addVar(lb=0)
         for tr in trains for (o,d,_) in demand[tr]}

t_arr = {(tr,n): m.addVar(lb=0,ub=T,vtype=gp.GRB.INTEGER)
         for tr in trains for n in nodes - {SINK}}

# ───────────────────────── 2. 제약
# (2) x ≤ h
for tr in trains:
    r_o   = routes_nodes[tr][0]        # 물리적 출발역
    t_dep = 0                          # or dep_time[tr]  (여기선 0)
    out_vars = [x[tr, fr, to, eid]
                for (fr, to, eid, _) in arc_list
                if fr == f"{r_o}^{t_dep}"]    # r_o^t_dep → j^{t_dep+τ}

    m.addConstr(gp.quicksum(out_vars) == h[tr],
                name=f"startFlow_{tr}")

# (3) 출발지 순유출
for tr in trains:
    r_o = routes_nodes[tr][0]
    out = gp.quicksum(v for (tr_,fr,*_),v in x.items()
                      if tr_==tr and fr.startswith(f"{r_o}^"))
    m.addConstr(out == h[tr])

# (4★) 종착 후보 선택 & dummy-arc 1개 연결
for tr in trains:
    m.addConstr(gp.quicksum(y[tr,n] for n in r_T[tr]) == h[tr])
    for n in r_T[tr]:
        dummy_sum = gp.quicksum(v for (tr_,fr,to,eid),v in x.items()
                                if tr_==tr and eid==f"dummy_{n}")
        m.addConstr(dummy_sum == y[tr,n])            # ★

# (5) 모든 노드( SINK 제외 ) 시간별 흐름보존
for tr in trains:
    r_o = routes_nodes[tr][0]
    for n in nodes - {SINK}:
        for t in range(T+1):
            nt = f"{n}^{t}"
            inflow  = gp.quicksum(v for (tr_,fr,to,*_),v in x.items()
                                  if tr_==tr and to == nt)
            outflow = gp.quicksum(v for (tr_,fr,to,*_),v in x.items()
                                  if tr_==tr and fr == nt)
            if n==r_o and t==0:        # 출발 노드·시각 = 순유출 = h
                m.addConstr(outflow - inflow == h[tr])
            elif n!=SINK:
                m.addConstr(inflow == outflow)

# (7’)(8’) 실제 in-arc 방문
for (tr, n), var_s in s.items():
    r_o = routes_nodes[tr][0]
    if n == r_o:
        real_flow = gp.quicksum(v for (tr_, fr, to, eid), v in x.items()
                                if tr_ == tr and fr.split("^")[0] == n
                                   and not eid.startswith(("w_", "dummy_")))
    else:
        real_flow = gp.quicksum(v for (tr_, fr, to, eid), v in x.items()
                                if tr_ == tr and to.split("^")[0] == n
                                   and not eid.startswith(("w_", "dummy_")))
    m.addConstr(real_flow >= var_s)
    m.addConstr(real_flow <= BIG_M * var_s)

# (10)~(12)+(15)
for tr in trains:
    for (o,d,_) in demand[tr]:
        m.addConstr(z[tr,o,d] <= s[tr,o])
        m.addConstr(z[tr,o,d] <= s[tr,d])
        m.addConstr(z[tr,o,d] >= s[tr,o] + s[tr,d] - 1)
        m.addConstr(z[tr,o,d] <= h[tr])

# 시간 링크 (dummy 제외)
for (tr,fr,to,eid),v in x.items():
    if eid.startswith(("w_","dummy_")): continue
    node_to, t_to = to.split("^"); t_to=int(t_to)
    m.addConstr(t_arr[tr,node_to] >= t_to - BIG_M*(1-v))
    m.addConstr(t_arr[tr,node_to] <= t_to + BIG_M*(1-v))

# Δ
for tr in trains:
    for (o,d,q) in demand[tr]:
        sched_t = sched[tr].get(d,T)
        m.addConstr(delta[tr,o,d] >= t_arr[tr,d]-sched_t - BIG_M*(1-z[tr,o,d]))
        m.addConstr(delta[tr,o,d] <= BIG_M*z[tr,o,d])

# (6) 용량 (dummy/wait 제외)
for eid,(src,dst,tau) in edges.items():
    if eid in failed_edges: continue
    for tt in range(T):
        flow = gp.quicksum(v for (tr,fr,to,e),v in x.items()
                           if e==eid and
                              int(fr.split("^")[1])<=tt<int(to.split("^")[1]))
        m.addConstr(flow <= CAPACITY)

# ───────────────────────── 3. 목적식
obj  = gp.quicksum(w1*q_r[tr]*(1-h[tr]) for tr in trains)
obj += gp.quicksum(w2*q*(1-z[tr,o,d])
                   for tr in trains for (o,d,q) in demand[tr])
obj += gp.quicksum(w3*q*delta[tr,o,d]*z[tr,o,d]
                   for tr in trains for (o,d,q) in demand[tr])
m.setObjective(obj, gp.GRB.MINIMIZE)

m.optimize()

# ───────────────────────── 4. 시간표 출력
rows=[]
for tr in trains:
    if h[tr].X<0.5:
        rows.append([tr,'-','-','-','-','Cancelled']); continue
    legs=[(int(fr.split("^")[1]),fr.split("^")[0],
           to.split("^")[0],int(to.split("^")[1]),eid)
          for (tr_,fr,to,eid),v in x.items()
          if tr_==tr and v.X>0.5 and not eid.startswith(("w_","dummy_"))]
    legs.sort(key=lambda k:k[0])
    for dep,frm,to,arr,eid in legs:
        rows.append([tr,frm,dep,to,arr,eid])

df=pd.DataFrame(rows,columns=["Train","From","Dep","To","Arr","Edge"])
print("\n=== Timetable (chronological) ===")
print(df.to_string(index=False))

# ───────────────────────── 5-A. objective breakdown
cancel_pen = sum(w1*q_r[tr]*(1-h[tr].X) for tr in trains)

unmet_pen  = sum(w2*q*(h[tr].X - z[tr,o,d].X)
                 for tr in trains for (o,d,q) in demand[tr])

delay_pen  = sum(w3*q*delta[tr,o,d].X*z[tr,o,d].X
                 for tr in trains for (o,d,q) in demand[tr])

print(f"\n=== Objective breakdown ===")
print(f"  Cancellation penalty : {cancel_pen:,.0f}")
print(f"  Un-served demand     : {unmet_pen:,.0f}")
print(f"  Delay penalty        : {delay_pen:,.0f}")
print(f"--------------------------------")
print(f"  Total objective      : {m.ObjVal:,.0f}")


# ───────────────────────── 5-B. train-level status
status_rows = []
for tr in trains:
    if h[tr].X < 0.5:
        status = "Cancelled"
    else:
        delayed = any(delta[tr,o,d].X > 1e-6 for (o,d,_) in demand[tr])
        unmet   = any(z[tr,o,d].X < 0.5 for (o,d,_) in demand[tr])
        if delayed:
            status = "Delayed"
        elif unmet:
            status = "Truncated/Rerouted"
        else:
            status = "On-time & complete"
    status_rows.append([tr, status])

df_stat = pd.DataFrame(status_rows, columns=["Train", "Status"])
print("\n=== Train status summary ===")
print(df_stat.to_string(index=False))



=== Timetable (chronological) ===
Train From  Dep To  Arr Edge
   T1   n1    0 n2    1   e1
   T1   n2    1 n3    2   e2
   T2   n1    0 n2    1   e1
   T2   n2    1 n4    2  e18
   T2   n4    2 n3    4   e4
   T3   n1    0 n5    2   e5
   T3   n5    2 n6    3   e6
   T3   n6    3 n7    4   e7
   T3   n7    4 n3    5   e8

=== Objective breakdown ===
  Cancellation penalty : 0
  Un-served demand     : 0
  Delay penalty        : 0
--------------------------------
  Total objective      : 0

=== Train status summary ===
Train             Status
   T1 On-time & complete
   T2 On-time & complete
   T3 On-time & complete


### Active set strategy

In [3]:
# ─────────────────── 0. 공통 입력 & 전처리 ───────────────────
import gurobipy as gp, pandas as pd
from collections import defaultdict
import itertools, math

# ---------- 0-a. 고정 파라미터 ----------
T          = 10
failed_edges = {"e3", "e12"}
max_wait   = 2
w1, w2, w3 = 1000, 50, 10
CAPACITY   = 3
BIG_M      = 10**6
SINK       = "SINK"

# ---------- 0-b. 네트워크 ----------
edges = {
    "e1": ("n1","n2",1), "e2": ("n2","n3",1),
    "e3": ("n1","n4",2), "e4": ("n4","n3",2),
    "e5": ("n1","n5",2), "e6": ("n5","n6",1),
    "e7": ("n6","n7",1), "e8": ("n7","n3",1),
    "e9": ("n4","n2",1), "e10":("n2","n1",1),
    "e11":("n3","n2",1), "e12":("n4","n1",2),
    "e13":("n3","n4",2), "e14":("n5","n1",2),
    "e15":("n6","n5",1), "e16":("n7","n6",1),
    "e17":("n3","n7",1), "e18":("n2","n4",1),
}
routes_nodes = {
    "T1":["n1","n2","n3"],
    "T2":["n1","n4","n3"],
    "T3":["n1","n5","n6","n7","n3"],
}
dep_time = {tr:0 for tr in routes_nodes}      # 출발시점 0 으로 단순화
demand   = {
    "T1":[("n1","n2",30),("n1","n3",50),("n2","n3",80)],
    "T2":[("n1","n4",50),("n1","n3",60),("n4","n3",100)],
    "T3":[("n1","n5",800),("n1","n6",60),("n1","n7",75),
          ("n1","n3",100),("n5","n6",25),("n5","n7",35),
          ("n5","n3",70),("n6","n7",15),("n6","n3",25),
          ("n7","n3",35)],
}
trains   = list(routes_nodes)
nodes    = {n for _, (s,d,_) in edges.items() for n in (s,d)}
nodes.add(SINK)

# ---------- 0-c. 시간 확장 아크 ----------
arc_list = []               # (from, to, eid, τ, start_t, end_t)
for eid,(src,dst,tau) in edges.items():
    if eid in failed_edges: continue
    for t in range(T + 1 - tau):
        arc_list.append((f"{src}^{t}", f"{dst}^{t+tau}", eid, tau, t, t+tau))
for n in nodes - {SINK}:
    for w in range(1, max_wait+1):
        for t in range(T + 1 - w):
            arc_list.append((f"{n}^{t}", f"{n}^{t+w}", f"w_{n}_{w}", w, t, t+w))
for n in nodes - {SINK}:
    for t in range(T+1):
        arc_list.append((f"{n}^{t}", f"{SINK}^{t}", f"dummy_{n}", 0, t, t))
# index ↔ arc 매핑
arc_idx = {info:i for i,info in enumerate(arc_list)}

# ---------- 0-d. OD 총 수요 ----------
q_r = {tr:sum(q for *_,q in demand[tr]) for tr in trains}

# ---------- 0-e. 예정 도착시각 (sched) ----------
sched={}
for tr, path in routes_nodes.items():
    t=0; arr={path[0]:dep_time[tr]}
    for i in range(len(path)-1):
        u,v = path[i],path[i+1]
        eid = next(e for e,(s,d,_) in edges.items() if s==u and d==v)
        t+=edges[eid][2]; arr[v]=dep_time[tr]+t
    sched[tr]=arr

# =====================================================================
# ① build_base_model – “항상 넣는” 제약만 가진 모델
# =====================================================================
def build_base_model():
    m = gp.Model(); m.Params.OutputFlag=0
    # vars
    x = m.addVars(len(arc_list), len(trains), vtype=gp.GRB.BINARY, name="x")
    h = m.addVars(trains, vtype=gp.GRB.BINARY, name="h")
    y = {(tr,n): m.addVar(vtype=gp.GRB.BINARY,name=f"y_{tr}_{n}")
         for tr in trains for n in routes_nodes[tr][1:]}
    s = {(tr,n): m.addVar(vtype=gp.GRB.BINARY,name=f"s_{tr}_{n}")
         for tr in trains for n in nodes - {SINK}}
    z = {(tr,o,d): m.addVar(vtype=gp.GRB.BINARY,name=f"z_{tr}_{o}_{d}")
         for tr in trains for (o,d,_) in demand[tr]}
    delta = {(tr,o,d): m.addVar(lb=0,name=f"del_{tr}_{o}_{d}")
         for tr in trains for (o,d,_) in demand[tr]}
    t_arr = {(tr,n): m.addVar(lb=0,ub=T,vtype=gp.GRB.INTEGER,
                              name=f"tarr_{tr}_{n}")
             for tr in trains for n in nodes - {SINK}}
    # --- start-flow (Equalities) ---
    for tr in trains:
        r_o = routes_nodes[tr][0]
        dep = dep_time[tr]
        out_vars=[x[arc_idx[(f"{r_o}^{dep}", to, eid, tau, dep, dep+tau)], tr_i]
                  for (f, to, eid, tau, _, _), tr_i in
                  ((info, trains.index(tr)) for info in arc_list)
                  if f==f"{r_o}^{dep}"]
        m.addConstr(gp.quicksum(out_vars)==h[tr], name=f"start_{tr}")
    # --- 노드 흐름보존 + dummy 처리 ---
    for tr in trains:
        tr_i = trains.index(tr)
        r_o  = routes_nodes[tr][0]
        dep  = dep_time[tr]
        for n in nodes - {SINK}:
            for t in range(T+1):
                idx_in  =[arc_idx[a] for a in arc_list if a[1]==f"{n}^{t}"]
                idx_out =[arc_idx[a] for a in arc_list if a[0]==f"{n}^{t}"]
                inflow  = x.sum(idx_in, tr_i)
                outflow = x.sum(idx_out, tr_i)
                if n==r_o and t==dep:
                    m.addConstr(outflow - inflow == h[tr])
                else:
                    m.addConstr(inflow == outflow)
    # --- y & dummy 연결 ---
    for tr in trains:
        tr_i=trains.index(tr)
        cand = routes_nodes[tr][1:]
        m.addConstr(gp.quicksum(y[tr,n] for n in cand)==h[tr])
        for n in cand:
            dummy_idx = [arc_idx[a] for a in arc_list if a[2]==f"dummy_{n}"]
            m.addConstr(x.sum(dummy_idx,tr_i)==y[tr,n])
    # --- node visit s ---
    for (tr,n), var in s.items():
        tr_i=trains.index(tr)
        idx = [arc_idx[a] for a in arc_list
               if (a[2].startswith("w_") or a[2].startswith("dummy"))==False]
        in_idx  = [k for k in idx if arc_list[k][1].split("^")[0]==n]
        out_idx = [k for k in idx if arc_list[k][0].split("^")[0]==n]
        flow = x.sum(in_idx,tr_i) + x.sum(out_idx,tr_i)
        m.addConstr(flow >= var)
        m.addConstr(flow <= BIG_M * var)
    # --- z / delta 논리 ---
    for tr in trains:
        for (o,d,q) in demand[tr]:
            m.addConstr(z[tr,o,d] <= s[tr,o])
            m.addConstr(z[tr,o,d] <= s[tr,d])
            m.addConstr(z[tr,o,d] >= s[tr,o] + s[tr,d] - 1)
            m.addConstr(z[tr,o,d] <= h[tr])
            # t_arr linkage later
    for tr in trains:
        tr_i=trains.index(tr)
        for k, (fr,to,eid,_,_,_) in enumerate(arc_list):
            if eid.startswith(("w_","dummy")): continue
            node_to = to.split("^")[0]; t_to=int(to.split("^")[1])
            m.addConstr(t_arr[tr,node_to] >= t_to - BIG_M*(1-x[k,tr_i]))
            m.addConstr(t_arr[tr,node_to] <= t_to + BIG_M*(1-x[k,tr_i]))
        for (o,d,q) in demand[tr]:
            sched_t = sched[tr].get(d,T)
            m.addConstr(delta[tr,o,d] >= t_arr[tr,d]-sched_t - BIG_M*(1-z[tr,o,d]))
            m.addConstr(delta[tr,o,d] <= BIG_M*z[tr,o,d])
    # --- 목적식 (penalty) ---
    obj  = gp.quicksum(w1*q_r[tr]*(1-h[tr]) for tr in trains)
    obj += gp.quicksum(w2*q*(1-z[tr,o,d])
                       for tr in trains for (o,d,q) in demand[tr])
    obj += gp.quicksum(w3*q*delta[tr,o,d]*z[tr,o,d]
                       for tr in trains for (o,d,q) in demand[tr])
    m.setObjective(obj, gp.GRB.MINIMIZE)
    return m, x, h

# =====================================================================
# ② 용량(부등식) 제약 후보 전부 사전 구축
# =====================================================================
# map: (eid,tt) -> [ (varIndex, trainIdx) ... ]
cap_map = defaultdict(list)
for k,(fr,to,eid, tau, t0, t1) in enumerate(arc_list):
    if eid.startswith(("w_","dummy")): continue
    for tt in range(t0, t1):          # arc 점유 구간
        cap_map[(eid,tt)].append(k)

# =====================================================================
# ③ 외부 active-set 루프
# =====================================================================
working_set = set()           # W₀  (비워 두고 시작)
sol_prev    = None
MAX_ITER    = 30
EPS         = 1e-6

for it in range(MAX_ITER):
    # ③-a 모델 생성
    base, xvar, hvar = build_base_model()
    # 용량제약 추가
    constr_refs = {}     # (eid,tt) -> constr object
    for key in working_set:
        expr = gp.quicksum(xvar[idx, ti]
               for idx in cap_map[key]
               for ti in range(len(trains)))   # 각 idx 는 모든 열차 변수 중 동일
        constr_refs[key] = base.addConstr(expr <= CAPACITY,
                                          name=f"cap_{key[0]}_{key[1]}")
    # Warm-start
    if sol_prev is not None:
        for (idx,tr_i), val in sol_prev.items():
            xvar[idx,tr_i].Start = val
    base.optimize()
    if base.Status != gp.GRB.OPTIMAL:
        raise RuntimeError("MIP infeasible/timeout")

    # ③-c 위반 탐색
    viol = set()
    x_val = {(idx,tr_i): round(xvar[idx,tr_i].X)
             for idx in range(len(arc_list))
             for tr_i in range(len(trains))}
    for key, idx_list in cap_map.items():
        flow = sum(x_val[idx,tr_i]
                   for idx in idx_list
                   for tr_i in range(len(trains)))
        if flow > CAPACITY + EPS:
            viol.add(key)

    # ③-d 음의 dual(π) 탐색
    neg_pi = set()
    lp = base.relax(); lp.optimize()
    if lp.Status == gp.GRB.OPTIMAL:
        for key, c in constr_refs.items():
            if c.Pi < -EPS:
                neg_pi.add(key)

    # ③-e 업데이트
    updated = False
    for key in viol:
        if key not in working_set:
            working_set.add(key); updated=True
    for key in neg_pi:
        if key in working_set:
            working_set.remove(key); updated=True

    ws_sorted = sorted(working_set)              # (eid,tt) 정렬
    preview   = ws_sorted[:MAX_ITER]             # 너무 길면 앞부분만
    print(f"[Iter {it}] |W|={len(working_set)}  preview={preview}")
    if len(ws_sorted) > MAX_ITER:
        print(f"          …(truncated, showing first {MAX_ITER})")

    # 저장된 해( Warm-start용 )
    sol_prev = x_val

    print(f"[Iter {it}]  |W|={len(working_set)},  new+{len(viol)}, drop−{len(neg_pi)}")
    if not updated:
        print("↳ converged.")
        break
else:
    print("⚠ Max-iter reached.")

# =====================================================================
# ④ 결과 리포트 (기존 출력 루틴 재활용)
# =====================================================================
rows=[]
for tr_i,tr in enumerate(trains):
    if hvar[tr].X < 0.5:
        rows.append([tr,'-','-','-','-','Cancelled']); continue
    legs=[(int(arc_list[idx][0].split("^")[1]),
           arc_list[idx][0].split("^")[0],
           arc_list[idx][1].split("^")[0],
           int(arc_list[idx][1].split("^")[1]),
           arc_list[idx][2])
          for idx in range(len(arc_list))
          if xvar[idx,tr_i].X > 0.5 and
             not arc_list[idx][2].startswith(("w_","dummy"))]
    legs.sort(key=lambda k:k[0])
    for dep,frm,to,arr,eid in legs:
        rows.append([tr,frm,dep,to,arr,eid])
df = pd.DataFrame(rows, columns=["Train","From","Dep","To","Arr","Edge"])
print("\n=== Timetable (chronological) ===")
print(df.to_string(index=False))


[Iter 0] |W|=0  preview=[]
[Iter 0]  |W|=0,  new+0, drop−0
↳ converged.

=== Timetable (chronological) ===
Train From  Dep To  Arr Edge
   T1   n1    0 n2    1   e1
   T1   n2    1 n3    2   e2
   T2   n1    0 n2    1   e1
   T2   n2    1 n4    2  e18
   T2   n4    2 n3    4   e4
   T3   n1    0 n5    2   e5
   T3   n5    2 n6    3   e6
   T3   n6    3 n7    4   e7
   T3   n7    4 n3    5   e8
